In [3]:
import numpy as np
import cv2

#### フレームレートの確認

In [4]:
# Webカメラを起動
cap = cv2.VideoCapture(0)

# フレームレートを取得
fps = cap.get(cv2.CAP_PROP_FPS)
print(f"フレームレート: {fps} fps")

# カメラを解放
cap.release()

フレームレート: 30.0 fps


### Pyramidal Lucas-kanade法

In [5]:
# 特徴点検出するためのパラメータ
# maxCorners=10: 検出する最大のコーナー（特徴点）の数を指定する。この場合、最大で10個の特徴点を検出する。
# qualityLevel=0.3: 特徴点の品質を決定するためのパラメータである。0から1の範囲で、値が小さいほど多くの特徴点が検出されるが、品質が低くなる可能性がある。
# minDistance=7: 検出された特徴点の間の最小距離を指定する。この距離より近い特徴点は無視される。これにより、重複した特徴点を避けることができる。
# blockSize=7: コーナー検出に使用する近傍のサイズを指定する。これは、特徴点の計算に使用されるピクセルのブロックのサイズである。

corner_track_params = dict(maxCorners=10, qualityLevel=0.3, minDistance=7, blockSize=7)

In [6]:
# lucas kanade法のパラメータ
# winSize = (200, 200): ウィンドウのサイズを指定する。ウィンドウは、特徴点の周囲の領域。その領域内で動きを推定する。(200, 200)は、ウィンドウの幅と高さをピクセル単位で示しており、200ピクセル×200ピクセルの領域が使用される。
# 大きなウィンドウサイズは、小さい動きを見落とす可能性が高く、一方、小さいウィンドウサイズは、大きい動きを見落とす可能性が高い。つまりトレードオフ。
# maxLevel = 2:
# このパラメータは、画像の解像度を指定する。https://en.wikipedia.org/wiki/Pyramid_(image_processing)を参考に。
# criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03):
# このパラメータは、オプティカルフローの計算を終了するための条件を指定する。
# cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT: 終了条件のタイプを指定する。ここでは、2つの条件を組み合わせている。
# cv2.TERM_CRITERIA_EPS: 精度が指定された閾値以下になると終了する。（精度）
# cv2.TERM_CRITERIA_COUNT: 最大反復回数に達すると終了する。（スピード）

lk_params = dict(winSize = (200,200), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10,0.03))

In [7]:
# PCのカメラを起動
cap = cv2.VideoCapture(0)

# PCのカメラを起動した際の最初のフレームを取得する
ret, prev_frame = cap.read()

# グレースケール画像を取得する（これを前のフレームとして参照する）
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# 特徴点を取得する（Shi-Tomasiコーナー検出）
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask = None, **corner_track_params)

# 後で描画するために、前のフレームのマスクを作成する（480, 640, 3)
mask = np.zeros_like(prev_frame)

while True:
    
    # 現在のフレームを取得する
    ret, frame = cap.read()
    
    # グレースケールを取得する
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # グレースケールフレームでオプティカルフローを計算する
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params)
    
    # 返されたステータス配列を使用する(10, 1, 2)
    # ステータス出力ベクトル（符号なし文字型）；ベクトルの各要素は、対応する特徴のフローが見つかった場合は1に設定され、そうでない場合は0に設定される。
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]
    
    for i, (new, prev) in enumerate(zip(good_new, good_prev)):
        
        x_new, y_new = new.ravel()
        x_prev, y_prev = prev.ravel()
        
        # 最初のフレームから作成したマスクを使用して線を描く
        mask = cv2.line(mask, (x_new, y_new), (x_prev, y_prev), (0, 255, 0), 3)
        
        # 特徴点に赤い円を描く
        frame = cv2.circle(frame, (x_new, y_new), 8, (0, 0, 255), -1)
    
    # 描いた線を含むマスクと画像を表示する
    img = cv2.add(frame, mask)
    cv2.imshow('frame', img)

    # Escで終了する
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
   
    # 前のフレームと前のポイントを更新する
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1, 1, 2)
    
cv2.destroyAllWindows()
cap.release()

### Farneback法

#### calcOpticalFlowFarneback(prev, next, flow, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags) -> flow
この関数は、Gunnar Farnebackのアルゴリズムを使用して密なオプティカルフローを計算します。以下は、関数のパラメータとその意味です：

- prev: 最初の8ビット単一チャンネル入力画像。
- next: prevと同じサイズおよび同じタイプの2番目の入力画像。
- flow: prevと同じサイズで、タイプがCV_32FC2の計算されたフロー画像。
- pyr_scale: 各画像のピラミッドを構築するための画像スケールを指定するパラメータ（1未満）。pyr_scale=0.5は、古典的なピラミッドを意味し、次の層は前の層の2倍小さくなります。
- levels: 初期画像を含むピラミッドの層数。levels=1の場合、追加の層は作成されず、元の画像のみが使用されます。
- winsize: 平均化ウィンドウのサイズ。大きい値は、画像のノイズに対するアルゴリズムの堅牢性を高め、高速な動きの検出の機会を増やしますが、よりぼやけた運動場を生じます。
- iterations: 各ピラミッドレベルでのアルゴリズムの反復回数。
- poly_n: 各ピクセルでの多項式展開を見つけるために使用されるピクセル近傍のサイズ。大きい値は、画像がより滑らかな表面で近似されることを意味し、より堅牢なアルゴリズムとよりぼやけた運動場を生じます。通常、poly_n=5または7。
- poly_sigma: 多項式展開の基盤として使用される導関数の平滑化に使用されるガウス分布の標準偏差。poly_n=5の場合、poly_sigma=1.1を設定できます。poly_n=7の場合、適切な値はpoly_sigma=1.5です。

In [11]:
import cv2 
import numpy as np

# フレームをキャプチャする
cap = cv2.VideoCapture(0)
ret, frame1 = cap.read()

# 最初のフレームのグレースケール画像を取得し、HSVカラーでマスクを作成する
prvsImg = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255

while True:
    ret, frame2 = cap.read()
    nextImg = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    
    flow = cv2.calcOpticalFlowFarneback(prvsImg,nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    
    # 移動の角度に基づいてチャンネルを色付けする
    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1],angleInDegrees=True)
    hsv_mask[:,:,0] = ang/2
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    # cvのimshowで表示するためにBGRに変換する
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame2',bgr)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    
    # ループのために前の画像を次の画像として設定する
    prvsImg = nextImg

cap.release()
cv2.destroyAllWindows()